<a href="https://colab.research.google.com/github/Nickmescher/ml5sem/blob/main/lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
sms_spam_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sms_spam.csv')
sms_spam_df.sample(10)

,type,text
1116,ham,Eh u send wrongly lar...
715,ham,Hi im having the most relaxing time ever! we h...
2363,ham,I dont want to hear anything
3319,ham,It's Ã© only $140 ard...Ã‰ rest all ard $180 a...
3961,ham,Well you told others you'd marry them...
177,ham,Oh thanks a lot..i already bought 2 eggs ..
432,ham,How to Make a girl Happy? It's not at all diff...
2613,ham,Havent still waitin as usual... u come back sc...
2035,spam,Dear Matthew please call 09063440451 from a la...
2463,ham,"Aight, I'll text you when I'm back"


In [ ]:
print('Missing values: ', sms_spam_df.isnull().values.any())
print('Duplicates: ', sms_spam_df.duplicated().any())

Missing values:  False
Duplicates:  True


In [ ]:
sms_spam_df.drop_duplicates(inplace=True)

In [ ]:
# приведение к нижнему регистру
sms_spam_df.text = sms_spam_df.text.str.lower()

In [ ]:
# токенизация

import nltk 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer()

stop_words = stopwords.words("english")


sms_spam_df.text = sms_spam_df.text.apply(lambda text: ' '.join([word
                                                                 for word in tweet_tokenizer.tokenize(text)]))

In [ ]:
# удаление пунктуации

import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
sms_spam_df.text = sms_spam_df.text.str.translate(str.maketrans('', '', string.punctuation)) \
                                   .str.strip(' ') \
                                   .str.replace(r' +', ' ')

In [ ]:
# лемматизация

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
sms_spam_df.text = sms_spam_df.text.apply(lambda text: ' '.join([str(token.lemma_)
                                                                 if token.lemma_ != '-PRON-'
                                                                 else str(token)
                                                                 for token in nlp(text)]))

In [ ]:
# удаление стоп слов

In [ ]:
sms_spam_df.text = sms_spam_df.text.apply(lambda text: ' '.join([word
                                                                 for word in tweet_tokenizer.tokenize(text)
                                                                 if word not in stop_words]))

In [ ]:
# векторизация

from sklearn.metrics import * 
from sklearn.model_selection import train_test_split 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(sms_spam_df.text,
                                                    sms_spam_df.type,
                                                    train_size = 0.7)

In [ ]:
assessments = {'precision': [],
               'recall': [],
               'f1-score': [],
               'accuracy': []}

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.feature_extraction.text import CountVectorizer 

In [ ]:
multinomial_nb_model = MultinomialNB()

In [ ]:
# мешок n-грамм

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

count_vectorizer = CountVectorizer()
bag_of_ngrams_pipeline = Pipeline([('count_vectorizer', count_vectorizer),
                                   ('multinomial_nb', multinomial_nb_model)])

param_grid = {'count_vectorizer__ngram_range': [(1, 2), (2, 2), 
                                                (2, 3), (3, 3), 
                                                (3, 4), (4, 4), 
                                                (1, 3), (1, 4)],
              'multinomial_nb__alpha': [0, 0.01, 0.1, 0.5, 1]}
gridsearch_cv = GridSearchCV(estimator=bag_of_ngrams_pipeline,
                             param_grid=param_grid,
                             n_jobs=-1,
                             cv=5)
gridsearch_cv.fit(x_train, y_train)
print(f'Best params: {gridsearch_cv.best_params_}')

metrics = classification_report(y_test,
                                gridsearch_cv.predict(x_test),
                                output_dict=True)

assessments['precision'].append(metrics['macro avg']['precision'])
assessments['recall'].append(metrics['macro avg']['recall'])
assessments['f1-score'].append(metrics['macro avg']['f1-score'])
assessments['accuracy'].append(metrics['accuracy'])

Best params: {'count_vectorizer__ngram_range': (1, 4), 'multinomial_nb__alpha': 0.5}


In [ ]:
# TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_pipeline = Pipeline([('tfidf', tfidf_vectorizer),
                           ('multinomial_nb', multinomial_nb_model)])

param_grid = {'tfidf__ngram_range': [(1, 2), (2, 2), 
                                     (2, 3), (3, 3), 
                                     (3, 4), (4, 4), 
                                     (1, 3), (1, 4)],
              'tfidf__max_df': np.linspace(0.01, 0.5, num = 5),
              'tfidf__min_df': np.linspace(0, 0.5, num = 5),
              'tfidf__max_features': [6000, 12000, 20000, 30000],
              'multinomial_nb__alpha': [0, 0.01, 0.1, 0.5, 1]}
gridsearch_cv = GridSearchCV(estimator=tfidf_pipeline,
                             param_grid=param_grid,
                             n_jobs=-1,
                             cv=5)
gridsearch_cv.fit(x_train, y_train)
print(f'Best params: {gridsearch_cv.best_params_}')

metrics = classification_report(y_test,
                                gridsearch_cv.predict(x_test),
                                output_dict=True)

assessments['precision'].append(metrics['macro avg']['precision'])
assessments['recall'].append(metrics['macro avg']['recall'])
assessments['f1-score'].append(metrics['macro avg']['f1-score'])
assessments['accuracy'].append(metrics['accuracy'])

Best params: {'multinomial_nb__alpha': 0.01, 'tfidf__max_df': 0.1325, 'tfidf__max_features': 6000, 'tfidf__min_df': 0.0, 'tfidf__ngram_range': (1, 2)}


In [ ]:
# Символьные n-граммы

In [ ]:
char_vectorizer = CountVectorizer(analyzer='char')

char_pipeline = Pipeline([('char_vectorizer', char_vectorizer),
                          ('multinomial_nb', multinomial_nb_model)])

param_grid = {'char_vectorizer__ngram_range': [(1, 2), (2, 2), 
                                               (2, 3), (3, 3), 
                                               (3, 4), (4, 4), 
                                               (1, 3), (1, 4)],
              'multinomial_nb__alpha': [0, 0.001, 0.01, 0.1, 1]}
gridsearch_cv = GridSearchCV(estimator=char_pipeline,
                             param_grid=param_grid,
                             n_jobs=-1,
                             cv=5)
gridsearch_cv.fit(x_train, y_train)
print(f'Best params: {gridsearch_cv.best_params_}')

metrics = classification_report(y_test,
                                gridsearch_cv.predict(x_test),
                                output_dict=True)

assessments['precision'].append(metrics['macro avg']['precision'])
assessments['recall'].append(metrics['macro avg']['recall'])
assessments['f1-score'].append(metrics['macro avg']['f1-score'])
assessments['accuracy'].append(metrics['accuracy'])

Best params: {'char_vectorizer__ngram_range': (2, 3), 'multinomial_nb__alpha': 0.1}


In [ ]:
assessments = pd.DataFrame(assessments, index =['Bag of N-grams',
                                                'TF-IDF',
                                                'Symbol N-grams',])

assessments

,precision,recall,f1-score,accuracy
Bag of N-grams,0.971571,0.971571,0.971571,0.988365
TF-IDF,0.970055,0.963191,0.966589,0.986425
Symbol N-grams,0.948787,0.957474,0.953073,0.980608


In [ ]:
# Из данной таблицы мы видим, что наибольшие значения метрик у векторайзера Bag of N-grams, соответственно можем сделать вывод о его
# наибольшой эффективности, а у 'Symbol of N-grams' в свою очередь самые маленькие значения среди всех векторайзеров
# Векторайзер 'TF-IDFs' сохраняет среднее положение между двумя остальными

## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **а?** - ноль или один символ **а**
* **а+** - один или более символов **а**
* **а\*** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа


In [ ]:
result = re.findall('a?b.', 'aabbсabbcbb') 
print(result)

['abb', 'abb', 'bb']


In [ ]:
result = re.findall('a*b.', 'aabbсabbcbb') 
print(result)

['aabb', 'abb', 'bb']


In [ ]:
result = re.findall('a+b.', 'aabbсabbcbb') 
print(result)

['aabb', 'abb']


Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

In [ ]:
# Потому что пересекается с ранее найденным совпадением

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
re.findall(r'\b[А-ЯЁа-яё][А-ЯЁа-яё]', 'Поставьте, пожалуйста, максимум баллов')

['По', 'по', 'ма', 'ба']

### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
result = re.split(r'(?<=\w[.!?])', 'Очень? Сильно! Люблю... Машинное. Обучение. И всё. Что с ним. Связано.', maxsplit=2)
print(result)

['Очень?', ' Сильно!', ' Люблю... Машинное. Обучение. И всё. Что с ним. Связано.']


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
result = re.sub(r'\d', 'DIG', 'Мой номер телефона легко запомнить 8-923-123-09-89')
print(result)

Мой номер телефона легко запомнить DIG-DIGDIGDIG-DIGDIGDIG-DIGDIG-DIGDIG


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
result = re.sub(r'http[s]?://[\w.:?/=]+', '', 'Ссылка на этот коллаб https://colab.research.google.com/drive/1NclA5kXO4snol50lz3zMGPf_SgbhEzjd?usp=sharing')
print(result)

Ссылка на этот коллаб 


### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog = re.compile(r'\w{4,}')
prog.findall("Мне одному кажется, что эта работа заслуживает 12 баллов?")

['одному', 'кажется', 'работа', 'заслуживает', 'баллов']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
prog = re.compile(r"@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b")
prog.findall('abc.test@gmail.com, xyz@te-st.in, test.first@analyticsvidhya.com, first.test@rest.biz')

['@gmail.com', '@te-st.in', '@analyticsvidhya.com', '@rest.biz']